In [2]:
import hope
hope.config.optimize = True
hope.config.verbose = True
hope.config.keeptemp = True
import numba
import numpy as np
from util import perf_comp_data
from native_util import load
%load_ext Cython
%load_ext version_information
%version_information numpy, Cython, numba, hope

Software versions
Python 3.5.1 64bit [GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
IPython 6.1.0
OS Darwin 15.6.0 x86_64 i386 64bit
numpy 1.13.1
Cython 0.26
numba 0.34.0
hope 0.6.1
Mon Sep 04 15:15:37 2017 CEST

In [3]:
# Pure python version

def pairwise_python(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)

In [4]:
# Numpy python version

def pairwise_numpy(X, D):
    M = X.shape[0]
    for i in range(M):
        D[i, :] = np.sqrt(np.sum((X[i, :] - X[:]) ** 2, axis=1))

In [5]:
# numba version
@numba.jit(nopython=True)
def pairwise_numba(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)

In [6]:
%%cython

cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X, double[:, ::1] D):
    cdef size_t M = X.shape[0]
    cdef size_t N = X.shape[1]
    cdef double tmp, d
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)

In [7]:
# hope version
@hope.jit
def pairwise_hope(X, D, M, N):
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)


In [8]:
from native_util import load

native_pairwise_mod = load("pairwise")
pairwise_native = native_pairwise_mod.run

running build_ext



In [9]:
X = np.random.random((1000, 3))
D = np.empty((1000, 1000))

In [10]:
D1 = np.empty((1000, 1000))
D2 = np.empty((1000, 1000))
D3 = np.empty((1000, 1000))
D4 = np.empty((1000, 1000))
D5 = np.empty((1000, 1000))
D6 = np.empty((1000, 1000))

pairwise_python(X, D1)
pairwise_numpy(X, D2)
pairwise_numba(X, D3)
pairwise_cython(X, D4)
pairwise_hope(X, D5, X.shape[0], X.shape[1])
pairwise_native(X, D6, X.shape[0], X.shape[1])

assert np.allclose(D1, D2)
assert np.allclose(D1, D3)
assert np.allclose(D1, D4)
assert np.allclose(D1, D5)
assert np.allclose(D1, D6)


pairwise_hope(float64^2 X, float64^2 D, int64 M, int64 N)
	for i.l in (0.J:M.J) {
		for j.l in (0.J:M.J) {
			new d.D
			d.D = 0.0.D
			for k.l in (0.J:N.J) {
				new tmp.d
				tmp.d = (X.d[i.l, k.l] - X.d[j.l, k.l])
				d.D += (tmp.d * tmp.d)
			}
			D.d[i.l, j.l] = numpy.sqrt(d.D)
		}
	}

Compiling following functions:
pairwise_hope(float64^2 X, float64^2 D, int64 M, int64 N)
running build_ext
building 'pairwise_hope_f82cb8a6ea3aecbde9f728fa85088b51f5bdf7d8ef8b740b9c0b788a_0' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11 -Wno-unreachable-code'
clang: /var/

In [12]:
print("naive python")
%timeit pairwise_python(X, D)
print("numpy")
%timeit pairwise_numpy(X, D)
print("numba")
%timeit pairwise_numba(X, D)
print("cython")
%timeit pairwise_cython(X, D)
print("hope")
%timeit pairwise_hope(X, D, X.shape[0], X.shape[1])
print("native")
%timeit pairwise_native(X, D, X.shape[0], X.shape[1])

naive python
3.99 s ± 64.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
numpy
42 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
numba
4.63 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
cython
4.75 ms ± 82.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
hope
6.02 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
native
4.72 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
from util import perf_comp_data

In [14]:
M, N = X.shape
data_list = 4*["X, D"]+ 2*["X, D, M, N"]
#print data_list
perf_comp_data(["pairwise_python", 
                "pairwise_numpy", 
                "pairwise_numba", 
                "pairwise_cython", 
                "pairwise_hope",
                "pairwise_native"],
               data_list, rep=100)

function: pairwise_numba      , av. time sec:   0.00433872, min. time sec:   0.00417360, relative:       1.0
function: pairwise_cython     , av. time sec:   0.00436183, min. time sec:   0.00415240, relative:       1.0
function: pairwise_native     , av. time sec:   0.00436626, min. time sec:   0.00414733, relative:       1.0
function: pairwise_hope       , av. time sec:   0.00550070, min. time sec:   0.00512893, relative:       1.3
function: pairwise_numpy      , av. time sec:   0.03921036, min. time sec:   0.03764664, relative:       9.0
function: pairwise_python     , av. time sec:   4.04625443, min. time sec:   3.89136448, relative:     932.6
